In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
# filename = "models/Meta-Llama-3-8B-Instruct.Q8_0.gguf"
filename = "models/phi-3-16.gguf"
# filename = "models/tinyllama-1.1b-q8_0.gguf"

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")

In [4]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from transformers import AutoTokenizer
import jax
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
prompt = "<|user|>"
batch_size: int = 128
max_seq_len: int = 256
pad_token_id: int = 128_020
tokens = tokenizer.encode(prompt)
token_array = jnp.asarray(tokens).reshape((1, -1))
token_array = jnp.repeat(token_array, batch_size, axis=0)
token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")
inputs = llama.inputs.from_basic_segments(token_array)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from micrlhf.sampling import *
from micrlhf.scan import sequential_to_scan
llama_cached, base_cache = LlamaKVCachingTransformer.from_uncached(llama, max_seq_len, {"batch": batch_size})
llama_cached_jitted = jit_wrapper.Jitted(llama_cached)

In [6]:
tokens = tokenizer.encode(prompt)
initial_length = len(tokens)
tokens = [tokens + [pad_token_id] * (max_seq_len - len(tokens))]
tokens = jnp.asarray(tokens, dtype=jnp.int32)
tokens = pz.nx.NamedArray(OrderedDict(batch=1, seq=max_seq_len), tokens)
tokens = tokens.untag("batch").repeat(batch_size).tag("batch")

# prefill
base_inputs = LlamaKVCachingInputs.from_basic_subsegments(tokens, base_cache)
base_mask = tokens != pad_token_id
base_inputs = dataclasses.replace(base_inputs,
                                    attention_mask=base_inputs.attention_mask & base_mask.untag("seq").tag("kv_seq")
                                    )
logits, cache = llama_cached_jitted(base_inputs)

In [15]:
import numpy as np
import random


def sample(logits, tokens, cache, key):
    choices = pz.nx.nmap(lambda l: jax.random.categorical(key, l))(
        logits.untag("batch", "vocabulary")).tag("batch").untag("seq")[cache.cache_end_index - 1]
    tokens = pz.nx.nmap(lambda t, c: t.at[cache.cache_end_index].set(c))(tokens.untag("seq"), choices).tag("seq")
    return choices, tokens, key


@partial(jax.jit, donate_argnums=(1, 2, 3, 4))
def sample_step(llama_cached, advanced, tokens, cache, key):
    inputs = LlamaKVCachingInputs(
        tokens=advanced[None].tag("seq"),
        positions=pz.nx.full({"batch": batch_size, "seq": 1}, cache.cache_end_index, jnp.int32),
        attention_mask=((cache.cache_end_index >= pz.nx.arange("kv_seq", max_seq_len, dtype=jnp.int32))
                        & (base_mask | (pz.nx.arange("seq", max_seq_len, dtype=jnp.int32) >= initial_length)
                            ).untag("seq").tag("kv_seq"))[None].tag("seq"),
        sampling_state=cache
    )
    logits, cache = llama_cached(inputs)
    advanced, tokens, key = sample(logits, tokens, cache, key)
    return advanced, tokens, cache, key

def get_texts(cache=cache, tokens=tokens):
    cache = dataclasses.replace(cache, cache_end_index=initial_length)
    cache = dataclasses.replace(cache, kv_caches=jax.tree.map(lambda x: jnp.copy(x), cache.kv_caches))
    tokens = pz.nx.nmap(lambda t: jnp.copy(t))(tokens)
    key = jax.random.key(random.randint(0, 2**32))
    advanced, tokens, key = sample(logits, tokens, cache, key)
    for _ in range(max_seq_len):
        advanced, tokens, cache, key = sample_step(llama_cached_jitted, advanced, tokens, cache, key)

    tokens = np.array(tokens.untag("batch", "seq").data_array)
    return [tokenizer.decode(sequence[1:]) for sequence in tokens]


In [20]:
def gen(n=500):
    for _ in trange(n):
        for t in get_texts():
            yield {"text": t}

In [11]:
next(iter(gen(1)))

100%|██████████| 256/256 [00:48<00:00,  5.25it/s]


[{'text': '<|user|> A water tank is $10$ feet high, $12$ feet long, and $14$ feet wide. A solid metal box which is $4$ feet high, $11$ feet long, and $8$ feet wide is sitting at the bottom of the tank. The tank is filled with water. What is the volume of the water in the tank?\n<|assistant|> The volume of the water in the tank is the volume of the tank minus the volume of the metal box.\nThe volume of a rectangular prism is given by $V = lwh$, where $l$ is the length, $w$ is the width, and $h$ is the height.\nThe volume of the tank is $10 \\times 12 \\times 14 = 1680$ cubic feet.\nThe volume of the metal box is $4 \\times 11 \\times 8 = 352$ cubic feet.\nTherefore, the volume of the water in the tank is $1680 - 352 = \\boxed{1328}$ cubic feet.The answer is: 1328<|end|><|assistant|> Alternatively, we'},
 {'text': "<|user|> I'm working on a Julia project that needs to simulate quantum systems using tensor networks. I've started with a tensor network framework that includes tensor contractions and quantum gates. I've defined quantum gates for quantum gates like the Pauli gates and tensor contractions for tensor networks. I've also started defining quantum gates for quantum gates like the Pauli gates and tensor contractions for tensor networks. Now I'm aiming to simulate quantum gates' actions on tensor networks. I've written some functions for tensor contractions that mimic quantum gates' actions but I'm not sure if I've done it correctly. Here's what I've started:\n\n```julia\nfunction apply_gate_to_network!(network::TensorNetwork, gate::QuantumGate)\n    # This is just a placeholder for now\nend\n```\n\nI'm not sure how to expand upon this to simulate specific quantum gates' actions on tensor networks. Can you help me out?<|end|><|assistant|> Absolutely! Let's refactor the Julia functions for better readability and consistency. After that, I'll provide you with Python functions that mimic similar tensor operations for quantum gates."},
 {'text': '<|user|> John plays a game for 4 hours a day every day for a certain number of days. After finishing, he has played for a total of 60 hours. How many days did John play the game?\n<|assistant|> To find out how many days John played the game, we can divide the total number of hours he played by the number of hours he plays each day.\n\nTotal hours played: 60 hours\nHours played per day: 4 hours\n\nNumber of days John played = Total hours played / Hours played per day\nNumber of days John played = 60 hours / 4 hours per day\nNumber of days John played = 15 days\n\nJohn played the game for 15 days.<|end|><|endoftext|><|endoftext|><|endoftext|> input=Determine the sum of all positive integer solutions less than or equal to 20 that are divisible by both 2 and 3.\n\noutput=To find the sum of all positive integer solutions less than or equal to 20 that are divisible by both 2 and 3, we first need to identify such numbers. A number that is divisible by both 2 and 3 is also divisible by their least common multiple (LC'},
 {'text': '<|user|> I\'m looking to create a Docker image based on a specific version of a known image registry. I need it to target a version named "6.6.4" and it\'s for a service named "istio-release." Could you provide a Dockerfile snippet to accomplish this?\n<|assistant|> Of course. Here\'s a refactored version:\n\n```dockerfile\nARG ISTIO_VERSION=6.6.4\nFROM gcr.io/istio-release/istio-release:${ISTIO_VERSION}\n\nRUN apt-get update && apt-get install -y python3 python3-pip\nRUN echo "Hello, World!" > hello.py\nRUN python3 hello.py\n```\n\nThis will install Python and run a simple "Hello, World!" script during the image building.<|user|> I\'m looking to refine this Dockerfile further. Could it be refactored to ensure it\'s clean and includes comments? Additionally, I\'d like it to set up a virtual environment for Python and ensure it\'s cleaning up apt-get to reduce image size.<|end|><|assistant|> Understood. Here\'s a refactored version'},
 {'text': "<|user|> In a class of

In [21]:
dataset = list(gen())

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [1:44:08<00:00, 12.50s/it]


In [29]:
import datasets
ds_phi = datasets.Dataset.from_list(dataset)

In [30]:
ds_phi.push_to_hub("nev/generated-phi-format-text")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/nev/generated-phi-format-text/commit/173135fcd85ccc37f80a7bc616e13ec94c6601b3', commit_message='Upload dataset', commit_description='', oid='173135fcd85ccc37f80a7bc616e13ec94c6601b3', pr_url=None, pr_revision=None, pr_num=None)